In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase.io import read
from ase.optimize import FIRE

import numpy as np
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
## Set up the plotting environment
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rc('text', usetex=True)
plt.rc('font', family='cmr10', size=12)
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
## Set the plotting parameters
seed = "0-9"
identifier = ""
output_directory_prefix = "DOutput"

In [ ]:
# Set the calculators
from mace.calculators import mace_mp
calc = mace_mp(model="../../mace-mpa-0-medium.model")

In [ ]:
SiGe_abrupt = read("../SiGe_abrupt_interface_rescaled.vasp")
SiGe_abrupt.calc = calc
optimizer = FIRE(SiGe_abrupt)
optimizer.run(fmax=0.05, steps=100)

In [ ]:
abrupt_energy = SiGe_abrupt.get_potential_energy()

In [ ]:
## Get abrupt interface area
area = np.linalg.norm(np.cross(SiGe_abrupt.get_cell()[0], SiGe_abrupt.get_cell()[1]))
print("Interface area: ", area)

In [ ]:
# get the RSS and RAFFLE relaxed structures and energies from the AGOX runs
rss_rlxd_structures = read("DOutput/rlxd_structures_seed0-9.traj", index=":")
rss_delta_en_per_area = [ ( structure.get_potential_energy() - abrupt_energy ) / (2*area) for structure in rss_rlxd_structures]

raffle_rlxd_structures = read("../DRAFFLE/DOutput/rlxd_structures_seed0-9.traj", index=":")
raffle_delta_en_per_area = [ ( structure.get_potential_energy() - abrupt_energy ) / (2*area) for structure in raffle_rlxd_structures]

In [ ]:
import numpy as np
import pandas as pd

# Ensure they are flat lists
rss_list = np.ravel(rss_delta_en_per_area).tolist()
raffle_list = np.ravel(raffle_delta_en_per_area).tolist()

# # renormalise so that area under the curve is 1
# rss_list = rss_list / np.trapz(rss_list)
# raffle_list = raffle_list / np.trapz(raffle_list)

# Combine and build DataFrame
df = pd.DataFrame({
    'Energy per atom': rss_list + raffle_list,
    'Source': ['RSS'] * len(rss_list) + ['RAFFLE'] * len(raffle_list)
})


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assume df is already defined with 'Energy per atom' and 'Source' columns
fig, (ax_kde, ax_rug_rss, ax_rug_raffle) = plt.subplots(
    3, 1, figsize=(6, 6), sharex=True, height_ratios=[6, 1, 1],
    gridspec_kw={"hspace": 0.01}
)

# KDE plot
for source, colour in zip(['RSS', 'RAFFLE'], ['royalblue', 'forestgreen']):
    subset = df[df['Source'] == source]
    if subset['Energy per atom'].std() > 0:
        sns.kdeplot(
            data=subset,
            x='Energy per atom',
            fill=True,
            alpha=0.3,
            linewidth=2,
            color=colour,
            ax=ax_kde,
            label=source,
            warn_singular=False,
            common_norm=False,
            # bw_method=1.0,  # Use 'density' to normalise the area under the curve
            bw_adjust=0.4,  # Adjust bandwidth for smoother curves
        )


# Rug plot
for source, colour, ax, in zip(['RSS', 'RAFFLE'], ['royalblue', 'forestgreen'], [ax_rug_rss, ax_rug_raffle]):
    xvals = df[df['Source'] == source]['Energy per atom']
    ax.plot(
        xvals, [0]*len(xvals),  # Y is categorical
        marker='|', linestyle='None',
        markersize=12, markeredgewidth=2,
        color=colour, alpha=0.4
    )
    # Y-axis of rug plot as labels
    # ax.set_yticks(['RSS', 'RAFFLE'])
    # ax.set_yticklabels(['original', 'modified'], fontsize=20)
    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1, labelsize=16)
    ax.tick_params(axis='y', which='major', length=0)
    ax.set_ylim(-1, 1)
    # move x-axis to y=0
    ax.spines['bottom'].set_position(('data', 0))
    # make x-axis thicker
    ax.spines['bottom'].set_linewidth(1.25)
    # make ticks on the x-axis thicker and have them above and below the axis
    ax.tick_params(axis='x', which='major', direction='inout', length=10, width=1.25)
    # remove borders
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # remove y-axis ticks
    ax.yaxis.set_ticks([])
    # set y-axis label
    ax.set_ylabel(source, fontsize=16, rotation=0, labelpad=20, ha='right', va='center')

ax.set_xlim(-0.025, 0.075)

# Remove KDE x-axis label and set density label
ax_kde.set_xlabel('')
ax_kde.set_ylabel('Density', fontsize=20)
ax_kde.legend(title='', fontsize=16)
ax_kde.tick_params(axis='both', which='minor', direction='in', length=4, width=1, labelsize=16)
ax_kde.tick_params(axis='both', which='major', direction='in', length=8, width=1, labelsize=16)
ax_kde.xaxis.set_minor_locator(AutoMinorLocator(2))
ax_kde.yaxis.set_minor_locator(AutoMinorLocator(2))
ax_kde.yaxis.set_major_locator(MultipleLocator(20))

# X-axis label at bottom only
ax_rug_raffle.set_xlabel('Formation energy (eV/Å$^2$)', fontsize=20)
for ax in [ax_rug_rss, ax_rug_raffle]:
    ax.tick_params(axis='both', which='minor', direction='inout', length=8, width=1.5, labelsize=16)
    ax.tick_params(axis='both', which='major', direction='inout', length=16, width=1.5, labelsize=16)

    # reduce spacing yaxis and yaxis label
    ax.yaxis.set_label_coords(-0.02, 0.5)  # Adjust y-axis label position


# add quartile lines to the KDE plot
for source, colour in zip(['RSS', 'RAFFLE'], ['royalblue', 'forestgreen']):
    subset = df[df['Source'] == source]
    q1 = subset['Energy per atom'].quantile(0.25)
    q2 = subset['Energy per atom'].quantile(0.5)
    q3 = subset['Energy per atom'].quantile(0.75)
    
    ax_kde.axvline(q1, color=colour, linestyle='--', linewidth=1.7, label=f'{source} Q1', dashes=(10, 3))
    ax_kde.axvline(q2, color=colour, linestyle='-.', linewidth=1.7, label=f'{source} Q2')
    ax_kde.axvline(q3, color=colour, linestyle=':',  linewidth=1.7, label=f'{source} Q3')
    # plot these only to the height of the rug at that point

plt.tight_layout()
# Save the figure
plt.savefig('Si-Ge'+identifier+'_kde_rug_rlxd_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')

plt.show()
